In [ ]:
"EEG correlates of working memory performance in females"
suppression of alpha1 activity and with the increase of the midline theta activity
decrement of beta1 power
Beta2 power increased
At the level of individual differences, we found that the high
performance (HP) group was characterized by higher alpha rhythm power. The HP group demonstrated task-related
increment of theta power in the left anterior area and a gradual increase of theta power at midline area. In contrast,
the low performance (LP) group exhibited a drop of theta power in the most challenging condition. HP group was
also characterized by stronger desynchronization of beta1 rhythm over the left posterior area in the manipulation
condition. In this condition, beta2 power increased in the HP group over anterior areas, but in the LP group over
posterior areas.
 low beta band (~13–20 Hz)
    beta2 (~20–30 Hz) 
    lower (13–20 Hz) and
upper (25–30 Hz) beta

"the electrophysiological underpinnings of variation in verbal working memory capacity"
our data indicate a close relationship between frontal
midline theta, central beta activity and the executive components of WM
In contrast, alpha activity was not related to the individual differences

"Oscillatory brain activity and maintenance of verbal and visual working memory: A systematic review"
frontal midline theta enhanced 
Alpha asymmetry (left < right)
alpha activity 

"Temporally distinct oscillatory codes of retention and manipulation of verbal working memory"
alpha_beta_theta

In [3]:
def bandpower(data, sf, band, method='welch', window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Requires MNE-Python >= 0.14.

    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    sf : float
      Sampling frequency of the data.
    band : list
      Lower and upper frequencies of the band of interest.
    method : string
      Periodogram method: 'welch' or 'multitaper'
    window_sec : float
      Length of each window in seconds. Useful only if method == 'welch'.
      If None, window_sec = (1 / min(band)) * 2.
    relative : boolean
      If True, return the relative power (= divided by the total power of the signal).
      If False (default), return the absolute power.

    Return
    ------
    bp : float
      Absolute or relative band power.
    """
    from scipy.signal import welch
    from scipy.integrate import simps
    from mne.time_frequency import psd_array_multitaper

    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * sf
        else:
            nperseg = (2 / low) * sf

        freqs, psd = welch(data, sf, nperseg=nperseg)

    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                          normalization='full', verbose=0)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp

In [8]:
from neurodsp.utils.download import load_ndsp_data
import numpy as np
data = load_ndsp_data('D:\\jupyter_folder\\nurun_python_codes\\pre_Sb.npy')
# Set sampling rate, and create a times vector for plotting
fs = 250

In [13]:
# Multitaper delta power
bp = bandpower(data[1,:], fs, [0.5, 4], 'multitaper')
bp_rel = bandpower(data[1,:], fs, [0.5, 4], 'multitaper', relative=True)
print('Absolute delta power: %.3f' % bp)
print('Relative delta power: %.3f' % bp_rel)

# Delta-beta ratio
# One advantage of the multitaper is that we don't need to define a window length.
db = bandpower(data[1,:], fs, [0.5, 4], 'multitaper') / bandpower(data[1,:], fs, [12, 30], 'multitaper')
print('Delta/beta ratio (absolute): %.3f' % db)

Absolute delta power: 1871.564
Relative delta power: 0.033
Delta/beta ratio (absolute): 0.194
Delta/beta ratio (relative): 0.194


In [10]:
data.shape


(21, 38250)